# Limpieza

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

stopwords = set(stopwords.words('english'))

In [2]:
datos = pd.read_csv('dataset/final-train.csv')
datos.head()

,Summary,Text,Score
0,Can really notice when I am not drinking it,I have had hormone problems in the past couple...,5
1,"Okay, just not the best",This is a decent balsamic glaze/reducation.<br...,4
2,"goo source of fiber, not so much a source of c...",the cookie is moist and chewy and has a great ...,3
3,Great GF Staple,I have a 4 yr old and a 2 yr old that can't ea...,5
4,Coffee just doesn't taste right without Sweet ...,We have drunk our coffee for 45 years using th...,5


In [20]:
# Vemos la primera entrada de la columna 'text' y la columna 'summary'
print(datos['Text'][1])

This is a decent balsamic glaze/reducation.<br /><br />It has a faint yeasty taste that I don't care for...not so heavy to preclude using it again, but I have a different brand that I love love love, so this is just a back up.<br /><br />FYI try Blaze balsamic reduction/glaze.  It is superb.


In [4]:
print(datos['Summary'][0])

Can really notice when I am not drinking it


In [5]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426340 entries, 0 to 426339
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Summary  426323 non-null  object
 1   Text     426340 non-null  object
 2   Score    426340 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 9.8+ MB


In [27]:
def limpiar_datos(dataframe):
    dataframe = dataframe.copy().dropna()
    dataframe['FullText'] = dataframe['Summary'] + ' ' + dataframe['Text']
    dataframe['FullText'] = dataframe['FullText'].str.replace(r'<.*?>', '', regex=True)
    dataframe['FullText'] = dataframe['FullText'].str.replace(r'[^\w\s]', ' ', regex=True).str.lower().apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))

    dataframe = dataframe[['FullText','Score']]

    return dataframe

In [28]:
datos_limpio = limpiar_datos(datos)

In [29]:
print(datos_limpio['FullText'][1])

okay best decent balsamic glaze reducation faint yeasty taste care heavy preclude using different brand love love love back fyi try blaze balsamic reduction glaze superb


In [30]:
datos_limpio.head()

,FullText,Score
0,really notice drinking hormone problems past c...,5
1,okay best decent balsamic glaze reducation fai...,4
2,goo source fiber much source chocoate cookie m...,3
3,great gf staple 4 yr old 2 yr old eat gluten a...,5
4,coffee taste right without sweet n low drunk c...,5


## Entrenamiento